# A comparison of CAO points between 2019, 2020, and 2021.
***

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests as rq
import re

- How to load CAO points information from the CAO website.
- A detailed comparison of CAO points in 2019, 2020, and 2021 using pandas

In [2]:
# Pulling raw data from CAO - level 8 (2021) [1]
CAO_2021 = rq.get("http://www2.cao.ie/points/l8.php")


In [3]:
# Identifying if they pull correctly (Response [200] means its good.)
CAO_2021

<Response [200]>

In [21]:
# Compiling the regular expression code to find lines of courses. [2] There should be 949 of them. 
# Third group is ran using the 'OR' function i.e. #123* OR #123 OR # OR 123* etc.. [3]
# RE is almost there however not capturing courses with no points assigned. Needs amendment.
re_course = re.compile("([A-Z]{2}[0-9]{3})  (.*[^[0-9]{3}|[A-Z]{3}|[\*]|[#])  *([#][0-9]{3}[\*]|[#][0-9]{3}|[#]|[0-9]{3}|[0-9]{3}[\*]|[\*])  (.*)")
counter = 0  
path = "2021_CAO_DATA.csv"

#(.*[^#|[0-9]{3}|\*])  * ([#][0-9]{3}[\*]|[#][0-9]{3}|[#]|[0-9]{3}|[0-9]{3}[\*]|[\*]|[ *])  (.*)")

with open (path, "w") as f:  # Writing lines from RE to a CSV file. 

# Running a loop to find lines in the above RE. 
    for line in CAO_2021.iter_lines(): 
        decoded = line.decode('cp1252') # error in the standard used. ISO standard didn't reckonise character '\x96' on CM002.
        if re_course.fullmatch(decoded):
            CSV_2021 = re_course.sub(r"\1,\2,\3,\4", decoded)
            f.write(CSV_2021 + "\n")
            counter +=1
            
print ("\nNumber of level 8 courses in 2021: {}\n".format(counter))


Number of level 8 courses in 2021: 923



<br>
<br>

Initally used the code below however it did not return course with no points assigned eg DB503.  Nor did it return courses with round two offers only e.g. GC301

- if re.fullmatch("[A-Z]{2}[0-9]{3}  .*[0-9]{3}(\*)? *", line.decode('ISO-8859-1')):

# References
- [1] http://www.cao.ie/index.php?page=points&p=2021
- [2] https://www.w3schools.com/python/python_regex.asp
- [2] https://realpython.com/regex-python/
- [3] https://www.ocpsoft.org/tutorials/regular-expressions/or-in-regex/
- https://www.w3schools.com/python/python_functions.asp
- https://stackoverflow.com/questions/54496411/python-errortypeerror-findall-missing-1-required-positional-argument-stri
- https://www.w3schools.com/python/python_regex.asp
- https://realpython.com/regex-python/


***
# End